# Twitter API Call

## w210 Capstone
## Spring 2018

By: Vyas, Phat, Stacia, Kasey

Last update: 01/24/2018

## Using Twitter live streaming API

In [1]:
## Import twittersearch API
from TwitterSearch import * ## please `pip install TwitterSearch` if not yet installed 
#import TwitterSearch
import pandas as pd
import json ## to load json format into dict
import time ## to extract date/time from retrieved data
#import datetime ## to get current date/time## Authentication

### Twitter authentication

In [2]:
## twitter authentication
ts = TwitterSearch(
        consumer_key = 'X2SwXG1wIBLJiJRUunqIXyaNz',
        consumer_secret = 'j0mtZ5BvcNE26rdRrySprP3iYhiAglUn8qtzGP5Rw4WXqm1lV8',
        access_token = '41741727-9JoDDAAoTUo47z8Zo8uCQ1t5WLcpWnoQVSrImoB7q',
        access_token_secret = 'dhEDyLezeg22qClu1qrIf3pVadN3n9KOxK8q72bCwvG42'
        
#         consumer_key = "Vk9kCWPUw8KctdN4QGywZFpvd",
#         consumer_secret =  "CoPsToBCXDYWkaxoSnPf9qnNEPazxvHGDzexDzZ6KJxq9ZplQb",
#         access_token =  "820596649-FSomVy8OakTPYrCKrJQw2IfsUZXGk5Xp6kCzfbFt",
#         access_token_secret =  "c2jLc2HNQPKFGncaeNQzRonB4B9LOOKOU67lVcaOeEzYK"
     )

### Helper functions

In [3]:
def date_time_transform(date_time):
    date_time_deconstruct = time.strptime(date_time, "%a %b %d %H:%M:%S +0000 %Y")
    date_output = str(date_time_deconstruct.tm_year) + '/' + str(date_time_deconstruct.tm_mon) + '/' + str(date_time_deconstruct.tm_mday)
    time_output = str(date_time_deconstruct.tm_hour) + ':' + str(date_time_deconstruct.tm_min)
    return date_output, time_output

def date_time_transform2(date_time):
    date_time_deconstruct = time.strptime(date_time, "%a %b %d %H:%M:%S %Y")
    date_output = str(date_time_deconstruct.tm_year) + str(date_time_deconstruct.tm_mon) + str(date_time_deconstruct.tm_mday)
    time_output = str(date_time_deconstruct.tm_hour) + str(date_time_deconstruct.tm_min)
    return date_output, time_output

### Search by keyword w/ wrapper

In [4]:
def search_keyword(keyword_list):
    """
        arg: list of keywords. This list is used as AND for all keywords input, therefore limit to only necessary pairs
        return: list of tweets
    """
    try:
        tso = TwitterSearchOrder() # create a TwitterSearchOrder object
        tso.set_keywords(keyword_list) ## list of keywords. 
        tso.set_language('en') ## English tweets only
        tso.set_include_entities(False) ## and don't give us all those entity information ## and don't give us all those entity information
        tso.set_result_type('mixed') ## Include both popular and real time results in the response.

        ## to avoid rate-limitation
        sleep_for = 10 # sleep for 60 seconds
        last_amount_of_queries = 0 # used to detect when new queries are done

        list_output = [] ## store retrieved tweets and their metadata in a list object
        count = 0
        for tweet in ts.search_tweets_iterable(tso):
            ## get the first tweet and store all metadata for reference later
#             if count < 1:
#                 full_sample = [] 
#                 for key in tweet:
#                     full_sample.append([key, tweet[key]])

#             ## print 5 tweets to indicate sucessful loading
#             if count < 5:
#                 print( '@%s tweeted: %s | on %s' % ( tweet['user']['screen_name'], tweet['text'], tweet['created_at'] ) )
#                 print('---------------------------------------')
#                 count += 1

            ## extract and split date time into its individual varaiable
            date_output, time_output = date_time_transform(tweet['created_at'])        

            ## extract relevant information from the tweet metadata
            output = [
                #['created_at', tweet['created_at']] ## date/time of tweet
                ['date', date_output]
                ,['time', time_output]
                ,['id_str', tweet['id_str']] ## user id
                ,['text',tweet['text']] ## actual tweet
                ,['user_friends_count', tweet['user']['friends_count']]
                ,['user_followers_count', tweet['user']['followers_count']]
                ,['user_location', tweet['user']['location']]
                ,['user_screen_name', tweet['user']['screen_name']]
                ,['user_verified', tweet['user']['verified']] ## if account is a verified account
                ,['coordinates', tweet['coordinates']]
                ,['retweet_count', tweet['retweet_count']]
                ]
            list_output.append(output)

            ## to avoid rate-limitation
            current_amount_of_queries = ts.get_statistics()[0]
            if not last_amount_of_queries == current_amount_of_queries:
                last_amount_of_queries = current_amount_of_queries
                time.sleep(sleep_for)

        return list_output

    except TwitterSearchException as e: # take care of all those ugly errors if there are some
        print(e)
        return None

def search_keyword_wrapper(keywords):
    """
        arg: list of keywords and use search_keyword() function to retrieve tweets associated with each keyword
        return: export all retrieved tweets into a dataframe and save in csv format
    """
    
    tempt_list = [] ## temporary list for extracting tweets
    
    ## loop through the list of keywords and retrieve tweets related to each keyword
    for keyword in keywords:
        print("-----------------------------------")
        print("Keyword(s):",keyword)
        results = search_keyword(keyword)
        print("Numbers of tweets:",len(results))
        tempt_list.append(results)

    output = []
    for l in range(len(tempt_list)):
        for i in range(len(tempt_list[l])):
            tempt = []
            for j in range(len(tempt_list[l][i])):
                tempt.append(tempt_list[l][i][j][1])
            output.append(tempt)

    ## create columns names
    fields =  ['date', 'time', 'id_str', 'text', 'user_friends_count', 'user_followers_count', 'user_location'
               ,'user_screen_name', 'user_verified','coordinates', 'retweet_count']

    ## transform temp_list into data frame for exportation
    df_output = pd.DataFrame(data = output, columns = fields)

    ## get date/time for 
    date_retrieved, time_retrieved = date_time_transform2(time.ctime())
    file_name = '../data/tweets/search_keyword_' + str(date_retrieved) + '_' + str(time_retrieved) + '.csv'

    df_output.to_csv(file_name,  sep = ',', index = False)

    print('Successfully export file ', file_name)

In [13]:
keyword_list = [['price']
                ,['lend']
                ,['chew']
                ,['emphasis']
                ,['retirement']
                ,['system']
                ,['Sunday']
                ,['chase']
                ,['guest']
                ,['define']
               ]

search_keyword_wrapper(keyword_list)

-----------------------------------
Keyword(s): ['price']
Numbers of tweets: 1998
-----------------------------------
Keyword(s): ['lend']
Numbers of tweets: 1697
-----------------------------------
Keyword(s): ['chew']
Numbers of tweets: 1999
-----------------------------------
Keyword(s): ['emphasis']
Numbers of tweets: 24499
-----------------------------------
Keyword(s): ['retirement']
Numbers of tweets: 14799
-----------------------------------
Keyword(s): ['system']
Numbers of tweets: 11292
-----------------------------------
Keyword(s): ['Sunday']
Numbers of tweets: 1694
-----------------------------------
Keyword(s): ['chase']
Numbers of tweets: 55394
-----------------------------------
Keyword(s): ['guest']
Numbers of tweets: 2596
-----------------------------------
Keyword(s): ['define']
Numbers of tweets: 11196
Successfully export file  ../data/tweets/search_keyword_2018128_1932.csv


In [ ]:
keyword_list = [['#GRAMMYs']
                ,['#ProBowl']
                ,['#NHLAllStarGame']
                ,['#RoyalRumble']
                ,['#TwitterCourseTopics']
                ,['Jay-Z']
                ,['Harrison Smith']
                ,['Marquette']
                ,['IKEA']
                ,['Isaac Haas']
               ]

search_keyword_wrapper(keyword_list)

In [ ]:
keyword_list = [['trump']
            ,['clinton']
            ,['russia','trump']
            ,['election']
            ,['republican']
            ,['democrat']
            ,['#maga']
           ]

search_keyword_wrapper(keyword_list)

In [ ]:
## print out the full metada list for reference
#full_sample

### Search by username

In [7]:
def search_user(username):
    """
        arg: twitter username
        return: list of tweets
    """
    try:
        tuo = TwitterUserOrder(username) # create a TwitterUserOrder
        
        ## to avoid rate-limitation
#         sleep_for = 10 # sleep for 60 seconds
#         last_amount_of_queries = 0 # used to detect when new queries are done

        # start asking Twitter about the timeline
        list_output = [] ## store retrieved tweets and their metadata in a list object
#         count = 0
        for tweet in ts.search_tweets_iterable(tuo):
#             ## get the first tweet and store all metadata for reference later
#             if count < 1:
#                 full_sample = [] 
#                 for key in tweet:
#                     full_sample.append([key, tweet[key]])

#             ## print 5 tweets to indicate sucessful loading
#             if count < 5:
#                 print( '@%s tweeted: %s | on %s' % ( tweet['user']['screen_name'], tweet['text'], tweet['created_at'] ) )
#                 print('---------------------------------------')
#                 count += 1

            ## extract and split date time into its individual varaiable
            date_output, time_output = date_time_transform(tweet['created_at'])        

            ## extract relevant information from the tweet metadata
            output = [
                #['created_at', tweet['created_at']] ## date/time of tweet
                ['date', date_output]
                ,['time', time_output]
                ,['id_str', tweet['id_str']] ## user id
                ,['text',tweet['text']] ## actual tweet
                ,['user_friends_count', tweet['user']['friends_count']]
                ,['user_followers_count', tweet['user']['followers_count']]
                ,['user_location', tweet['user']['location']]
                ,['user_screen_name', tweet['user']['screen_name']]
                ,['user_verified', tweet['user']['verified']] ## if account is a verified account
                ,['coordinates', tweet['coordinates']]
                ,['retweet_count', tweet['retweet_count']]
                ]
            list_output.append(output)
            
            ## to avoid rate-limitation
#             current_amount_of_queries = ts.get_statistics()[0]
#             if not last_amount_of_queries == current_amount_of_queries:
#                 last_amount_of_queries = current_amount_of_queries
#                 time.sleep(sleep_for)
        
        return list_output

    except TwitterSearchException as e: # take care of all those ugly errors if there are some
        print(e)
        return None
    
def search_user_wrapper(user):
    user_results = search_user(user)
    len(user_results)
    
    output = []
    for i in range(len(user_results)):
        tempt = []
        for j in range(len(user_results[i])):
            tempt.append(user_results[i][j][1])
        output.append(tempt)

    ## create columns names
    fields =  ['date', 'time', 'id_str', 'text', 'user_friends_count', 'user_followers_count', 'user_location'
               ,'user_screen_name', 'user_verified','coordinates', 'retweet_count']

    ## transform temp_list into data frame for exportation
    df_output = pd.DataFrame(data = output, columns = fields)

    ## get date/time for 
    date_retrieved, time_retrieved = date_time_transform2(time.ctime())
    file_name = '../data/tweets/search_user_' + user + '_' + str(date_retrieved) + '_' + str(time_retrieved) + '.csv'

    df_output.to_csv(file_name,  sep = ',', index = False)

    print('Successfully export file ', file_name)

In [8]:
user = 'washingtonpost'
search_user_wrapper(user)

3202

## Search by UserID

In [5]:
def search_userid(userid):
    """
        arg: twitter userid
        return: list of tweets
    """
    try:
        tuo = TwitterUserOrder(userid) # create a TwitterUserOrder
        
        ## to avoid rate-limitation
        sleep_for = 10 # sleep for 60 seconds
        last_amount_of_queries = 0 # used to detect when new queries are done

        # start asking Twitter about the timeline
        list_output = [] ## store retrieved tweets and their metadata in a list object
#         count = 0
        for tweet in ts.search_tweets_iterable(tuo):
#             ## get the first tweet and store all metadata for reference later
#             if count < 1:
#                 full_sample = [] 
#                 for key in tweet:
#                     full_sample.append([key, tweet[key]])

#             ## print 5 tweets to indicate sucessful loading
#             if count < 5:
#                 print( '@%s tweeted: %s | on %s' % ( tweet['user']['screen_name'], tweet['text'], tweet['created_at'] ) )
#                 print('---------------------------------------')
#                 count += 1

            ## extract and split date time into its individual varaiable
            date_output, time_output = date_time_transform(tweet['created_at'])        

            ## extract relevant information from the tweet metadata
            output = [
                #['created_at', tweet['created_at']] ## date/time of tweet
                ['date', date_output]
                ,['time', time_output]
                ,['id_str', tweet['id_str']] ## user id
                ,['text',tweet['text']] ## actual tweet
                ,['user_friends_count', tweet['user']['friends_count']]
                ,['user_followers_count', tweet['user']['followers_count']]
                ,['user_location', tweet['user']['location']]
                ,['user_screen_name', tweet['user']['screen_name']]
                ,['user_verified', tweet['user']['verified']] ## if account is a verified account
                ,['coordinates', tweet['coordinates']]
                ,['retweet_count', tweet['retweet_count']]
                ]
            list_output.append(output)
            
            ## to avoid rate-limitation
            current_amount_of_queries = ts.get_statistics()[0]
            if not last_amount_of_queries == current_amount_of_queries:
                last_amount_of_queries = current_amount_of_queries
                time.sleep(sleep_for)
        
        return list_output

    except TwitterSearchException as e: # take care of all those ugly errors if there are some
        print(e)
        return None
    
def search_userid_wrapper(user):
    user_results = search_userid(user)
    print("User id:",str(user))
    len(user_results)
    
    output = []
    for i in range(len(user_results)):
        tempt = []
        for j in range(len(user_results[i])):
            tempt.append(user_results[i][j][1])
        output.append(tempt)

    ## create columns names
    fields =  ['date', 'time', 'id_str', 'text', 'user_friends_count', 'user_followers_count', 'user_location'
               ,'user_screen_name', 'user_verified','coordinates', 'retweet_count']

    ## transform temp_list into data frame for exportation
    df_output = pd.DataFrame(data = output, columns = fields)

    ## get date/time for 
    date_retrieved, time_retrieved = date_time_transform2(time.ctime())
    file_name = '../data/tweets/search_userid_' + str(user) + '_' + str(date_retrieved) + '_' + str(time_retrieved) + '.csv'

    df_output.to_csv(file_name,  sep = ',', index = False)

    print('Successfully export file ', file_name)

In [6]:
user = 3098421349
search_userid_wrapper(user)

User id: 3098421349
Successfully export file  ../data/tweets/search_userid_3098421349_201822_2123.csv


## Sandbox

In [ ]:
try:
    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(['trump'#,'russia','clinton','america','american'
                      #,'Trump','Russia','Clinton','America','American'
                      #,'election','democrats','republican'
                      #,'#Trump','#Russia','#Clinton','#American'
                     ]) ## list of keywords. This list is used as AND
    tso.set_language('en') ## English tweets only
    tso.set_include_entities(False) ## and don't give us all those entity information ## and don't give us all those entity information
    tso.set_result_type('mixed') ## Include both popular and real time results in the response.

    list_output = [] ## store retrieved tweets and their metadata in a list object
    count = 0
    for tweet in ts.search_tweets_iterable(tso):
        ## get the first tweet and store all metadata for reference later
        if count < 1:
            full_sample = [] 
            for key in tweet:
                full_sample.append([key, tweet[key]])

        ## print 5 tweets to indicate sucessful loading
        if count < 5:
            print( '@%s tweeted: %s | on %s' % ( tweet['user']['screen_name'], tweet['text'], tweet['created_at'] ) )
            print('---------------------------------------')
            count += 1

        ## extract and split date time into its individual varaiable
        date_output, time_output = date_time_transform(tweet['created_at'])        

        ## extract relevant information from the tweet metadata
        output = [
            #['created_at', tweet['created_at']] ## date/time of tweet
            ['date', date_output]
            ,['time', time_output]
            ,['id_str', tweet['id_str']] ## user id
            ,['text',tweet['text']] ## actual tweet
            ,['user_friends_count', tweet['user']['friends_count']]
            ,['user_location', tweet['user']['location']]
            ,['user_screen_name', tweet['user']['screen_name']]
            ,['user_verified', tweet['user']['verified']] ## if account is a verified account
            ,['coordinates', tweet['coordinates']]
            ,['retweet_count', tweet['retweet_count']]
        ]

        list_output.append(output)

except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)

In [4]:
## query to retrieve twwet
def search_keyword(keyword_list):
    """
        arg: list of keywords. This list is used as AND for all keywords input, therefore limit to only necessary pairs
        return: list of tweets
    """
    try:
        tso = TwitterSearchOrder() # create a TwitterSearchOrder object
        tso.set_keywords(keyword_list) ## list of keywords. 
        tso.set_language('en') ## English tweets only
        tso.set_include_entities(False) ## and don't give us all those entity information ## and don't give us all those entity information
        tso.set_result_type('mixed') ## Include both popular and real time results in the response.

        ## to avoid rate-limitation
        sleep_for = 10 # sleep for 60 seconds
        last_amount_of_queries = 0 # used to detect when new queries are done

        list_output = [] ## store retrieved tweets and their metadata in a list object
        count = 0
        for tweet in ts.search_tweets_iterable(tso):
            ## get the first tweet and store all metadata for reference later
#             if count < 1:
#                 full_sample = [] 
#                 for key in tweet:
#                     full_sample.append([key, tweet[key]])

#             ## print 5 tweets to indicate sucessful loading
#             if count < 5:
#                 print( '@%s tweeted: %s | on %s' % ( tweet['user']['screen_name'], tweet['text'], tweet['created_at'] ) )
#                 print('---------------------------------------')
#                 count += 1

            ## extract and split date time into its individual varaiable
            date_output, time_output = date_time_transform(tweet['created_at'])        

            ## extract relevant information from the tweet metadata
            output = [
                #['created_at', tweet['created_at']] ## date/time of tweet
                ['date', date_output]
                ,['time', time_output]
                ,['id_str', tweet['id_str']] ## user id
                ,['text',tweet['text']] ## actual tweet
                ,['user_friends_count', tweet['user']['friends_count']]
                ,['user_followers_count', tweet['user']['followers_count']]
                ,['user_location', tweet['user']['location']]
                ,['user_screen_name', tweet['user']['screen_name']]
                ,['user_verified', tweet['user']['verified']] ## if account is a verified account
                ,['coordinates', tweet['coordinates']]
                ,['retweet_count', tweet['retweet_count']]
                ]
            list_output.append(output)
            
            ## to avoid rate-limitation
            current_amount_of_queries = ts.get_statistics()[0]
            if not last_amount_of_queries == current_amount_of_queries:
                last_amount_of_queries = current_amount_of_queries
                time.sleep(sleep_for)

        return list_output

    except TwitterSearchException as e: # take care of all those ugly errors if there are some
        print(e)
        return None

In [5]:
keywords = [['trump']
            ,['clinton']
#             ,['russia','trump']
#             ,['election']
#             ,['republican']
#             ,['democrat']
            ,['#maga']
           ]

tempt_list = [] ## temporary list for extracting data
for keyword in keywords:
    print("-----------------------------------")
    print("Keyword(s):",keyword)
    results = search_keyword(keyword)
    print("Numbers of tweets:",len(results))
    tempt_list.append(results)

## Check if it's successful
print(len(tempt_list[0]))
print(tempt_list[0][1])

-----------------------------------
Keyword(s): ['trump']
Numbers of tweets: 699
-----------------------------------
Keyword(s): ['clinton']
Numbers of tweets: 228899
-----------------------------------
Keyword(s): ['#maga']
Numbers of tweets: 96
699
[['date', '2018/1/25'], ['time', '21:14'], ['id_str', '956636374228176900'], ['text', 'What about the 19 women who accused Trump of sexual assault and harassment? Are you inspired by their stories?  https://t.co/LmbpWWzaEn'], ['user_friends_count', 1489], ['user_followers_count', 20536], ['user_location', 'Brooklyn, NY'], ['user_screen_name', 'LEBassett'], ['user_verified', True], ['coordinates', None], ['retweet_count', 9158]]


In [6]:
output = []
for l in range(len(tempt_list)):
    for i in range(len(tempt_list[l])):
        tempt = []
        for j in range(len(tempt_list[l][i])):
            tempt.append(tempt_list[l][i][j][1])
        output.append(tempt)

## create columns names
fields =  ['date', 'time', 'id_str', 'text', 'user_friends_count', 'user_location', 'user_followers_count'
           ,'user_screen_name', 'user_verified','coordinates', 'retweet_count']

## transform temp_list into data frame for exportation
df_output = pd.DataFrame(data = output, columns = fields)

## get date/time for 
date_retrieved, time_retrieved = date_time_transform2(time.ctime())
file_name = '../data/search_keyword_' + str(date_retrieved) + '_' + str(time_retrieved) + '.csv'

df_output.to_csv(file_name,  sep = ',', index = False)

print('Successfully export file ', file_name)

Successfully export file  ../data/search_keyword_2018127_158.csv


In [7]:
keywords = [
             ['russian','investigation']    
            ,['election']
            ,['republican']
            ,['democrat']
           ]

tempt_list = [] ## temporary list for extracting data
for keyword in keywords:
    print("-----------------------------------")
    print("Keyword(s):",keyword)
    results = search_keyword(keyword)
    print("Numbers of tweets:",len(results))
    tempt_list.append(results)

## Check if it's successful
print(len(tempt_list[0]))
print(tempt_list[0][1])

-----------------------------------
Keyword(s): ['russian', 'investigation']
Numbers of tweets: 1190
-----------------------------------
Keyword(s): ['election']
Numbers of tweets: 27999
-----------------------------------
Keyword(s): ['republican']
Numbers of tweets: 79089
-----------------------------------
Keyword(s): ['democrat']
Numbers of tweets: 13289
1190
[['date', '2018/1/25'], ['time', '20:38'], ['id_str', '956627401357787137'], ['text', "For years, Dutch intelligence had access to Russia's Cozy Bear hackers group. Crucial evidence for Russian interfer… https://t.co/ENj8n2AzEa"], ['user_friends_count', 937], ['user_followers_count', 13010], ['user_location', '#standplaatslaptop'], ['user_screen_name', 'trbrtc'], ['user_verified', True], ['coordinates', None], ['retweet_count', 1616]]


In [8]:
output = []
for l in range(len(tempt_list)):
    for i in range(len(tempt_list[l])):
        tempt = []
        for j in range(len(tempt_list[l][i])):
            tempt.append(tempt_list[l][i][j][1])
        output.append(tempt)

## create columns names
fields =  ['date', 'time', 'id_str', 'text', 'user_friends_count', 'user_location', 'user_followers_count'
           ,'user_screen_name', 'user_verified','coordinates', 'retweet_count']

## transform temp_list into data frame for exportation
df_output = pd.DataFrame(data = output, columns = fields)

## get date/time for 
date_retrieved, time_retrieved = date_time_transform2(time.ctime())
file_name = '../data/search_keyword_' + str(date_retrieved) + '_' + str(time_retrieved) + '.csv'

df_output.to_csv(file_name,  sep = ',', index = False)

print('Successfully export file ', file_name)

Successfully export file  ../data/search_keyword_2018127_533.csv


In [7]:
## print out the first retrieved tweet and its metadata
results[5]

[['date', '2018/1/26'],
 ['time', '1:25'],
 ['id_str', '956699594091630593'],
 ['text',
  'RT @samstein: hahahaha. everyone on the Hill who has been like, Trump won’t try to fire mueller, he knows that would be a red line. https:/…'],
 ['user_friends_count', 741],
 ['user_location', 'Miami, FL'],
 ['user_screen_name', 'HDBtweets'],
 ['user_verified', False],
 ['coordinates', None],
 ['retweet_count', 74]]